### The following section includes sourced code to generate a graph coloring problem.

In [1]:
################################################################
##
## GRAPH COLORING PROBLEM GENERATOR
##
## Generates a planar graph.
##
################################################################

import random

class Point:
    ID_COUNT = 0
    def __init__(self, x, y):
        self.id = Point.ID_COUNT
        Point.ID_COUNT += 1
        self.x = x
        self.y = y

    def transform(self, xt, yt):
        return (xt(self.x), yt(self.y))

    def dist(self, them):
        return abs(self.x - them.x) + abs(self.y - them.y)

    def __repr__(self):
        return "p({:.4f}, {:.4f})".format(self.x, self.y)

## From http://bryceboe.com/2006/10/23/line-segment-intersection-algorithm/
def _ccw(A,B,C):
    return (C.y-A.y)*(B.x-A.x) > (B.y-A.y)*(C.x-A.x)

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    @property
    def endpoints(self):
        return [self.p1, self.p2]

    def transform(self, xt = lambda x: x, yt = lambda y: y):
        start = self.p1.transform(xt, yt)
        end = self.p2.transform(xt, yt)
        return (start, end)

    def intersects(self, them):
        A = self.p1
        B = self.p2
        C = them.p1
        D = them.p2
        return _ccw(A,C,D) != _ccw(B,C,D) and _ccw(A,B,C) != _ccw(A,B,D)

    def __repr__(self):
        return "[{} -> {}]".format(self.p1, self.p2)

def _random_point():
    return Point(random.uniform(-10, 10), random.uniform(-10, 10))

def _find_line(x, lines, pairs):
    for i, x1 in enumerate(x[:-1]):
        shortest_items = sorted([(x2, x1.dist(x2)) for x2 in x[i+1:]], key=lambda item: item[1])
        for x2, _ in shortest_items:
            l1 = Line(x1, x2)
            if not (x1, x2) in pairs and not _line_intersects(l1, lines):
                return l1
    return None

def _line_intersects(l1, lines):
    for l2 in lines:
        if l1.p1 in l2.endpoints or l1.p2 in l2.endpoints:
            continue
        if l1.intersects(l2): return True
    return False

def gen(num_points=100):
    x = [_random_point() for _ in range(num_points)]
    lines = set([])
    pairs = set([])
    while True:
        random.shuffle(x)
        line = _find_line(x, lines, pairs)
        if line:
            pairs.add((line.p1, line.p2))
            lines.add(line)
        else:
            break
    return (x, lines)

In [2]:
import json

num_points = 100
output_file = "gcp.json"

print("Generating a planar graph with {} points...".format(num_points))
(x, lines) = gen(num_points=num_points)

print("Writing to '{}'...".format(output_file))
with open(output_file, 'w') as f:
    f.write(json.dumps({
        'num_points': len(x),
        'points': { p.id: (p.x, p.y) for p in x },
        'edges': [ (line.p1.id, line.p2.id) for line in lines ]
    }, indent=2))
print("""Done! You can now import the results into a Python script with the code:

import json

with open('{}', \'r\') as f:
    data = json.load(f)
""".format(output_file))

Generating a planar graph with 100 points...
Writing to 'gcp.json'...
Done! You can now import the results into a Python script with the code:

import json

with open('gcp.json', 'r') as f:
    data = json.load(f)



In [3]:
import json

with open('gcp.json', 'r') as f:
    data = json.load(f)
    print(data)

{'num_points': 100, 'points': {'2': [7.767375578449656, 4.0630450560534666], '95': [5.589466170146768, -1.2936060910206297], '37': [1.7345782955621036, 4.615975021759976], '50': [3.8151024816210484, -6.08764354637221], '81': [4.797196531558098, -7.326968218286137], '18': [-9.747548384730502, 9.888591451487358], '22': [8.69210991912363, 4.813576477612667], '43': [8.53615594528123, -4.631812418624433], '74': [0.44139345125504725, 3.8692979643209124], '35': [-3.8973153944896666, -6.876474897187686], '12': [6.790297389052238, 3.3494248727855016], '56': [0.4409960009553764, 6.772417239773098], '97': [2.7932898819563245, 9.28666961227946], '94': [0.43101139937462385, 5.505483224630856], '45': [0.3848716462994304, -8.551842914462433], '30': [-2.713475183752438, 5.365495409815699], '27': [-8.390779844907687, -2.1927618774897883], '44': [-6.9111934895575144, -2.741966316885886], '88': [0.49637161237723504, 5.1104430878661695], '47': [4.458980904082566, -0.22434119362431382], '9': [-0.0174770251

### The following section includes sourced code to generate a Sudoku problem.

In [4]:
################################################################
## CLI Wrapper for a Sudoku generator
##  by Chad Crawford, using code from Gareth Rees
################################################################

import random
from functools import *

## From Gareth Rees at https://codereview.stackexchange.com/a/88866
def make_board(m: int=3) -> list[list[int]]:
    """Return a random filled m**2 x m**2 Sudoku board."""
    n = m**2
    board = [[None for _ in range(n)] for _ in range(n)]

    def search(c: int=0) -> list[list[int]]:
        "Recursively search for a solution starting at position c."
        i, j = divmod(c, n)
        i0, j0 = i - i % m, j - j % m # Origin of mxm block
        numbers = list(range(1, n + 1))
        random.shuffle(numbers)
        for x in numbers:
            if (x not in board[i]                     # row
                and all(row[j] != x for row in board) # column
                and all(x not in row[j0:j0+m]         # block
                        for row in board[i0:i])):
                board[i][j] = x
                if c + 1 >= n**2 or search(c + 1):
                    return board
        else:
            # No number is valid in this cell: backtrack and try again.
            board[i][j] = None
            return None

    return search()

In [5]:
import json

def write_board(cell_length: int, num_missing: int, output_file: str):
    """
    Create a Sudoku board with the given dimensions and write it to the inputted output file path.
    """
    board = make_board(cell_length)

    ms = int(pow(cell_length, 2))
    indices = random.sample(list(range(ms * ms)), num_missing)
    for index in indices:
        r = int(index / ms)
        c = int(index % ms)
        board[r][c] = 0

    ## Write to file
    with open(output_file, 'w') as f:
        f.write(json.dumps(board))

### The following section includes my own implementation of solving a constraint satisfaction problem.

In [6]:
# Before we get into solving a constraint satisfaction problem, we need to define one.
from enum import Enum
from queue import LifoQueue, PriorityQueue, Queue
from typing import Self
import copy

class Consistency(Enum):
    BACKTRACK = 1
    FORWARD_CHECK = 2
    AC3 = 3

class CSPHeuristic(Enum):
    RANDOM = 1 # Random variable is picked next
    MIN_REMAINING = 2 # Variable with the fewest remaining values to be assigned to it is picked next
    MIN_REMAINING_AND_DEGREE = 3 # Variable with the most other variables connected to it is picked next, with tie-breakers going to the fewest remaining available variables to assign

class CSPGraph:
    """
    The graph class will be responsible for remembering connections between nodes and maintaining the variables available for each node.
    In addition, when a node is assigned a value, this class is responsible for maintaining the variable options remaining for each node.
    This class is also going to be responsible for maintaining the specified k-consistency (backtrack for k=0, forward_check for k=1, ac3 for k=2) as each node is assigned a value
    """
    # The following function is mainly book-keeping for its format - the functions used to validate two sets of options between two adjacent nodes have this format
    @staticmethod
    def __consistency_checker_format(first_options: set[int], second_options: set[int]) -> set[int]:
        """
        Return all variables removed from the second set of options to preserve consistency
        """
        pass

    def __init__(self, cell_size: int, consistency: Consistency, consistency_checker: type[__consistency_checker_format], heuristic: CSPHeuristic):
        """
        When initializing a Graph object, all we need to know is the number of possible values that can be assigned to each of this graph's nodes.
        """
        self.__default_options = range(1, cell_size+1)
        self.__consistency = consistency
        self.__heuristic = heuristic
        self.__connections = {}
        self.__available_values = {}
        self.__assigned_values = {}
        self.__consistency_checker = consistency_checker
        self.__variables = []
        self.__record_assignments = 0
    
    def add_node(self, id: int):
        """
        Helper method to add a node to our underlying graph if it does not already exist
        """
        if id in self.__connections.keys():
            return
        self.__connections[id] = set()
        self.__available_values[id] = set()
        for value in self.__default_options:
            self.__available_values[id].add(value)
        self.__variables.append(id)

    def connect(self, first: int, second: int):
        """
        Connect those two nodes to each other in the underlying graph
        """
        self.__connections[first].add(second)
        self.__connections[second].add(first)

    def remove_option(self, id: int, option: int):
        """
        Method to remove an option from the input node's set of options if it is not already removed
        """
        if option in self.__available_values[id]:
            self.__available_values[id].remove(option)

    def solve_graph(self):
        """
        Start assigning variable different values until the CSP is solved
        """
        # Perform an initial sort on our variables, and then make the recursive call
        self.__variables.sort(key=self.__sort_criterion)
        self.__record_assignments = 0
        self.__recursive_assignment()
        
    def __recursive_assignment(self) -> bool:
        """
        Recursive helper method to assign all variables values while maintaining our desired consistency
        """
        i = 0
        while self.__variables[i] in self.__assigned_values.keys():
            i += 1
        next_id = self.__variables[i]
        # Now look at all the variable options for this variable
        initial_options = self.__copy_options(id=next_id)
        for option in initial_options:
            # Try assigning this value to the variable
            self.__assigned_values[next_id] = option
            # Clear out this node's options - its only possibility now is what we are assigning it
            self.__available_values[next_id] = set()
            self.__available_values[next_id].add(option)

            # For each id, remember the variables removed from it
            removed = {}
            if self.__update_available(id=next_id, removed=removed):
                if len(self.__assigned_values) > self.__record_assignments:
                    print(f"Record Variable Assignments Achieved: {len(self.__assigned_values)}")
                    self.__record_assignments = len(self.__assigned_values)
                # Our consistency checking has not stopped us yet
                # Resort our variables because they may have had their available options decreased in number
                self.__variables.sort(key=self.__sort_criterion)
                # Now we're ready for a recursive call
                if len(self.__assigned_values) == len(self.__connections) or self.__recursive_assignment():
                    # Complete assignment was achieved!
                    return True
                
            # Otherwise, restore all variables whose available values we modified
            for id, removed_options in removed.items():
                # Restore the removed options to each variable
                for option in removed_options:
                    self.__available_values[id].add(option)
        
        # If no options worked
        if next_id in self.__assigned_values.keys():
            del self.__assigned_values[next_id]
            # Restore this node's original options
            self.__available_values[next_id] = initial_options
        
        return False
    
    def __sort_criterion(self, id: int) -> int:
        """
        This is how we prioritize our variables when we assign them
        """
        if self.__heuristic == CSPHeuristic.RANDOM:
            return int(random.random()*10)
        elif self.__heuristic == CSPHeuristic.MIN_REMAINING:
            return len(self.__available_values[id])
        else:
            # MIN_REMAINING_AND_DEGREE - prioritize higher degree AN lower remaining
            return len(self.__available_values[id]) - len(self.__connections[id])

    def __copy_options(self, id: int) -> set[int]:
        """
        Helper method to copy the options associated with a particular variable
        """
        copy = set()
        for v in self.__available_values[id]:
            copy.add(v)
        return copy

    def __update_available(self, id: int, removed: dict[int, set[int]]) -> bool:
        """
        Given a node that has just been assigned a value, update all neighbors' available values according to our arc consistency, and return False if any problems occur.
        """
        # In all cases, we remove options from our immediate neighbors, but some consistencies check farther...
        if self.__consistency == Consistency.BACKTRACK:
            # We're not doing any kind of forward checking and we only run into a problem once we hit a node with no available values to assign to it
            for neighbor in self.__connections[id]:
                to_remove = self.__consistency_checker(self.__available_values[id], self.__available_values[neighbor])
                for var in to_remove:
                    self.__available_values[neighbor].remove(var)
                    if neighbor not in removed.keys():
                        removed[neighbor] = set()
                    removed[neighbor].add(var)
            return True
        elif self.__consistency == Consistency.FORWARD_CHECK:
            # Forward checking
            for neighbor in self.__connections[id]:
                to_remove = self.__consistency_checker(self.__available_values[id], self.__available_values[neighbor])
                for var in to_remove:
                    self.__available_values[neighbor].remove(var)
                    if neighbor not in removed.keys():
                        removed[neighbor] = set()
                    removed[neighbor].add(var)
                if len(self.__available_values[neighbor]) == 0:
                    # The neighbor RAN OUT of options to be assigned to it - the most previous variable assignment for the node of the input id did not work
                    return False
            return True
        else:
            # AC3 consistency
            edge_queue = Queue()
            for neighbor in self.__connections[id]:
                edge_queue.put((id, neighbor))
            while not edge_queue.empty():
                next_edge = edge_queue.get()
                current, neighbor = next_edge[0], next_edge[1]
                to_remove = self.__consistency_checker(self.__available_values[current], self.__available_values[neighbor])
                for var in to_remove:
                    self.__available_values[neighbor].remove(var)
                    if neighbor not in removed.keys():
                        removed[neighbor] = set()
                    removed[neighbor].add(var)
                if len(self.__available_values[neighbor]) == 0:
                    # The neighbor RAN OUT of options to be assigned to it - the most previous variable assignment for the node of the input id did not work
                    return False
                elif len(to_remove) > 0:
                    # Continue the algorithm
                    for other_neighbor in self.__connections[neighbor]:
                        if other_neighbor != current:
                            edge_queue.put((neighbor, other_neighbor))
            # The queue of edges emptied with no problems
            return True

    def get_assignment(self, id: int) -> int:
        """
        Retrieve the assignment for the variable with the given id
        """
        return int(self.__assigned_values[id]) if self.__assigned_values[id] != None else self.__assigned_values[id]

In [7]:
import json
from math import sqrt
import numpy as np

class SudokuSolver:
    """
    Solver for Sudoku Problem
    """
    
    def __init__(self, puzzle_file: str, consistency: Consistency, heuristic: CSPHeuristic):

        with open(puzzle_file, 'r') as f:
            rows = json.load(f)
            array = np.array(rows, dtype=int)
            # Turn the 2D sudoku board into a 1D array
            self.__underlying_values = array.flatten()
            # Store a value that equals the following three equivalent things: number of rows, number of columns, and number of cells
            self.__length = len(rows)
            # Store the indices that belong in each row
            self.__rows_by_indices = [[self.__length*i + j for j in range(self.__length)] for i in range(self.__length)]
            # Store the indices that belong in each column
            self.__cols_by_indices = [[self.__length*i + j for i in range(self.__length)] for j in range(self.__length)]
            # Store the indices that belong in each cell
            self.__cells_by_indices = []
            cell_length = int(sqrt(self.__length))
            for j in range(self.__length):
                current_row_indices = range(j//cell_length*cell_length, j//cell_length*cell_length+cell_length)
                current_col_indices = range((j % cell_length)*cell_length,(j % cell_length)*cell_length+cell_length)
                current_cell_indices = []
                for r_idx in current_row_indices:
                    for c_idx in current_col_indices:
                        current_cell_indices.append(self.__length*r_idx + c_idx)
                self.__cells_by_indices.append(current_cell_indices)
            # Initialize the graph
            self.__graph = CSPGraph(cell_size=self.__length, consistency=consistency, consistency_checker=self.__check_available, heuristic=heuristic)

    def solve(self):
        """
        Create an underlying CSPGraph to solve the problem.
        """
        # Display all the initial values
        for row in self.__rows_by_indices:
            print([int(self.__underlying_values[i]) for i in row])
        print('\n')

        for i in range(len(self.__underlying_values)):
            # If the value at this position is zero, then as far as we know all values are an option.
            if self.__underlying_values[i] == 0:
                self.__graph.add_node(id=i)
        
        # Connect all rows together
        for row in self.__rows_by_indices:
            self.__connect_all(row)

        # Connect all columns together
        for col in self.__cols_by_indices:
            self.__connect_all(col)
        
        # Connect all cells together
        for cell in self.__cells_by_indices:
            self.__connect_all(cell)

        # Let the graph do the solving
        self.__graph.solve_graph()

        # Now display all the assigned values
        for row in self.__rows_by_indices:
            print([self.__graph.get_assignment(id=i) if self.__underlying_values[i] == 0 else int(self.__underlying_values[i]) for i in row])

    def __connect_all(self, group: list[int]):
        """
        For a given row, column, or cell, ensure that the respective portion in the underlying graph is fully connected.
        """
        taken_values = set()
        free_spots = []
        for posn in group:
            if self.__underlying_values[posn] == 0:
                self.__graph.add_node(id=posn)
                free_spots.append(posn)
            else:
                taken_values.add(self.__underlying_values[posn])
        # Connect all free spots in the underlying graph
        for i in range(len(free_spots)-1):
            for j in range(i+1, len(free_spots)):
                self.__graph.connect(free_spots[i],free_spots[j])
        # Remove all taken_values from options of free_spots
        for spot in free_spots:
            for v in taken_values:
                self.__graph.remove_option(id=spot, option=v)

    def __check_available(self, first_values: set[int], second_values: set[int]) -> set[int]:
        """
        This is how a Sudoku Solver determines if any variable options from the second set need to be pruned.
        """
        remove = set() # If the first variable has only ONE option available for it, then the second variable CANNOT have that option in its set of options.
        if len(first_values) == 1:
            first_value = list(first_values)[0]
            if first_value in second_values:
                remove.add(first_value)
        return remove

#### We can now observe the performance of the above algorithm when trying different heuristics for ordering variables, as well as different consistencies being enforced as we solve.

In [8]:
# write_board(cell_length=4, num_missing=100, output_file="sudoku.json")

In [9]:
import time

print(f"Consistency: {Consistency.BACKTRACK.name}; Heuristic: {CSPHeuristic.RANDOM.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: BACKTRACK; Heuristic: RANDOM:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variable Assignme

In [10]:
import time

print(f"Consistency: {Consistency.BACKTRACK.name}; Heuristic: {CSPHeuristic.MIN_REMAINING.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: BACKTRACK; Heuristic: MIN_REMAINING:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variable A

In [11]:
import time

print(f"Consistency: {Consistency.BACKTRACK.name}; Heuristic: {CSPHeuristic.MIN_REMAINING_AND_DEGREE.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: BACKTRACK; Heuristic: MIN_REMAINING_AND_DEGREE:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record

In [12]:
import time

print(f"Consistency: {Consistency.FORWARD_CHECK.name}; Heuristic: {CSPHeuristic.RANDOM.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: FORWARD_CHECK; Heuristic: RANDOM:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variable Assi

In [13]:
import time

print(f"Consistency: {Consistency.FORWARD_CHECK.name}; Heuristic: {CSPHeuristic.MIN_REMAINING.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: FORWARD_CHECK; Heuristic: MIN_REMAINING:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variab

In [14]:
import time

print(f"Consistency: {Consistency.FORWARD_CHECK.name}; Heuristic: {CSPHeuristic.MIN_REMAINING_AND_DEGREE.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: FORWARD_CHECK; Heuristic: MIN_REMAINING_AND_DEGREE:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Re

In [15]:
import time

print(f"Consistency: {Consistency.AC3.name}; Heuristic: {CSPHeuristic.RANDOM.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: AC3; Heuristic: RANDOM:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variable Assignments Ac

In [16]:
import time

print(f"Consistency: {Consistency.AC3.name}; Heuristic: {CSPHeuristic.MIN_REMAINING.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: AC3; Heuristic: MIN_REMAINING:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Variable Assignm

In [17]:
import time

print(f"Consistency: {Consistency.AC3.name}; Heuristic: {CSPHeuristic.MIN_REMAINING_AND_DEGREE.name}:")
start_time = time.time()
sudoku_solver = SudokuSolver(puzzle_file="sudoku.json",consistency=Consistency.BACKTRACK, heuristic=CSPHeuristic.RANDOM.name)
sudoku_solver.solve()
end_time = time.time()
print(f"Time: {end_time - start_time} seconds")
print("\n=================================================================\n")

Consistency: AC3; Heuristic: MIN_REMAINING_AND_DEGREE:
[9, 0, 0, 14, 12, 0, 0, 0, 0, 6, 5, 11, 13, 0, 4, 2]
[0, 2, 3, 10, 6, 0, 1, 5, 0, 4, 16, 7, 12, 0, 15, 0]
[0, 0, 6, 15, 0, 0, 0, 0, 0, 13, 12, 0, 0, 3, 0, 1]
[7, 4, 5, 0, 13, 2, 15, 0, 0, 0, 1, 14, 0, 0, 0, 16]
[3, 12, 9, 5, 1, 13, 0, 14, 7, 10, 0, 0, 15, 2, 0, 0]
[10, 0, 14, 7, 0, 12, 4, 8, 16, 0, 2, 5, 11, 1, 6, 3]
[0, 16, 2, 0, 5, 6, 0, 15, 13, 0, 0, 1, 14, 12, 10, 8]
[1, 0, 15, 0, 10, 0, 0, 3, 0, 11, 14, 4, 0, 13, 9, 7]
[0, 5, 0, 9, 0, 14, 12, 13, 0, 1, 7, 16, 0, 6, 3, 15]
[0, 8, 0, 0, 2, 0, 0, 1, 3, 0, 0, 15, 9, 0, 11, 0]
[6, 15, 4, 3, 7, 10, 9, 16, 11, 0, 13, 8, 2, 5, 0, 14]
[0, 14, 13, 1, 0, 15, 0, 0, 9, 0, 10, 0, 0, 0, 7, 0]
[5, 3, 12, 13, 0, 9, 0, 0, 0, 0, 0, 0, 1, 16, 2, 11]
[14, 0, 1, 2, 15, 8, 10, 0, 6, 16, 0, 0, 4, 0, 13, 5]
[0, 10, 11, 4, 0, 5, 0, 2, 0, 0, 0, 12, 3, 0, 0, 6]
[0, 9, 0, 6, 14, 1, 3, 0, 5, 2, 0, 13, 0, 8, 0, 0]


Record Variable Assignments Achieved: 1
Record Variable Assignments Achieved: 2
Record Varia